In [1]:

from utils.trainfiles import trainfiles
from utils.dataloader import dataloader
from model.unet import UNet

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, random_split

if torch.cuda.is_available():
    print('GPU ready')
else:
    print('Warning: only CPU found')

GPU ready


In [2]:
trainfiles_ = trainfiles('trainfiles.json',False)
dataloader_ = dataloader(trainfiles_,4,4,train_height=400,train_width=400,load_multiple_targets_per_file=True,n_multiple_targets=4)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = UNet(4).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.MSELoss()


In [3]:
# Training loop
num_epochs = 1
for epoch in range(num_epochs):
    i = 0
    while not dataloader_.epoch_done:
        batch_generated = dataloader_.get_batch()
        if not batch_generated:
            break
        data = dataloader_.X.to(device)
        targets = dataloader_.y.to(device)
        model.train()
        outputs = model(data)
        loss = criterion(outputs, targets)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if i % 100 == 0:
            print(f'Batch {i+1}, Loss: {loss.item()}')
        i += 1
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')
    dataloader_.shuffle_array()

Batch 1, Loss: 0.0786810964345932


KeyboardInterrupt: 